# Example Notebook for Keysight Network Analyzers

This notebook is indendet to give an overview over the functions implemented in the QCoDeS driver for the Keysight network analyzers. The driver is implemented to be generic as possible, with individual instrument drivers filling in only the hardware limits of the instrument, so although this example uses the N5245A, the concepts and code should work for any keysight network analyzer.

In [1]:
%matplotlib inline

In [2]:
# Import Dependencies

import logging
import matplotlib.pyplot as plt

# qcodes imports
import qcodes as qc
from qcodes.dataset.data_set import new_data_set
from qcodes.dataset.experiment_container import new_experiment, load_experiment_by_name
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id
from qcodes.instrument_drivers.Keysight.N5222B import N5222B

In [3]:
qc.initialise_database()

# Start experiment
exp_name = 'PNA_Example_N5222B'
sample_name = 'sample_bar_code'
try:
    exp = load_experiment_by_name(exp_name, sample=sample_name)
    print('Experiment loaded. Last ID no:', exp.last_counter)
except ValueError:
    exp = new_experiment(exp_name, sample_name)
    print('Starting new experiment.')

Experiment loaded. Last ID no: 4


## Connect to the instrument

You will have to insert the correct VISA address for your PNA below. On my PC, the PNA has IP address `192.168.0.10`. You can generally use NI MAX or Agilent IO Toolkit to figure out what the VISA address of your instrument is, particularly for USB or GPIB connections.

In [4]:
pna = N5222B('pna', 'GPIB0::16::INSTR')

Connected to: Keysight Technologies N5222B (serial:MY57041028, firmware:A.13.25.04) in 0.16s


In [11]:
pna.channel[0].trace

{}

In [6]:
pna.channel[0].add_trace("TEST", "S11")

TEST, S11

In [8]:
pna.channel[1].trace

{}

In [9]:
pna.channel[0].trace["TEST"]

TEST, S11

In [10]:
pna.channel[0].trace["TEST"].delete()

In [6]:
pna.power(-40)
pna.start(100e6)
pna.stop(1e9)
pna.points(1001)

In [7]:
data = trace.log_magnitude()

NameError: name 'trace' is not defined

In [10]:
class A: 
    def __del__(self): 
        print("I am gone")
        
d = {"a": A()}

del d["a"]

print(d)

I am gone
{}


In [14]:
class A: 
    pass 

d = {"a": A()}
n = d["a"]
del n
print(d)

{'a': <__main__.A object at 0x000001A959EBD550>}
